Welcome to my Options Pricing and Volatility project. I will be implementing all the knowledge I can extract from the book in practical usable ideas.
- SSV 31 January 2025.

In [1]:
# Importing neccesary libraries.

import numpy as np
import matplotlib as pl
import pandas as pd
from tabulate import tabulate
from IPython.display import HTML
from fractions import Fraction

In [6]:
#### ================================================
# Contract settlement and cash flow calculator
#=================================================

""" 
Contract settlement and cash flow calculator for stock-type settlement.

Use cases:    
    - Shares belong to the same security.
    - Open long and short positions allowed.

The method used to manage the execution of trades is the FIFO (First in first out) which means the 
oldest shares are sold first. This method is the deafult one used by most exchanges and regulators

Additionally, an option to calculate interest earnt or lost is shown. It does so following
these assumptions:
    - Each trade is made the 1st day of the month.
    - There is no compounding of the interest (interest over the interest).
    - One month is exactly 1/12 of a year.

"""

# ================================================
# 1. Initialize global variables
#=================================================

trades = []
positions = pd.DataFrame(columns=["Stock price", "Long", "Short"])

open_share_position = 0
cashflow = 0
cum_cashflow = 0
realized_pnl = 0
unrealized_pnl = 0
total_cum_pnl = 0
trade_sequence = 0

# ================================================
# 2. Function to process trades
#=================================================

def process_trade(stock_price, quantity, action):

    """
    Processes the trade according to user inputs.

    Parameters:
    - stock_price (float): current market price of the stock.
    - quantity (int): number of shares bought/sold.
    - action (str): type of transaction performed, either "buy" or "sell".

    Notes:
    - A record of all positions long or short and their respective stock prices is kept for PnL calculations.

    """
    
    global open_share_position, cashflow, cum_cashflow, realized_pnl, unrealized_pnl, total_cum_pnl, trade_sequence, shares
    
    if action == 'buy':
        open_share_position += quantity 
        cashflow = -quantity * stock_price
    elif action == 'sell':
        open_share_position -= quantity
        cashflow = quantity * stock_price
    
    trade_sequence = trade_sequence + 1
    
    cum_cashflow += cashflow

    realized_pnl = update_positions(stock_price, quantity, action)
    
    unrealized_pnl = calculate_upnl(stock_price, quantity, action)

    total_cum_pnl += realized_pnl + unrealized_pnl

    trade = {
        "Trade sequence": trade_sequence,
        "Stock price": stock_price,
        "Trade": f"{'Buy' if action == 'buy' else 'Sell'} {abs(quantity)}",
        "Open share position": f"{'Long' if open_share_position >= 0 else 'Short'} {abs(open_share_position)}",
        "Cash flow credit (+) or debit (-)": cashflow,
        "Cumulative cash flow": cum_cashflow,
        "Realized PnL": realized_pnl,
        "Unrealized PnL": unrealized_pnl,
        "Total cumulative profit (+) or loss (-)": total_cum_pnl
    }

    trades.append(trade)
    
# ================================================
# 3. Function to update positions and calculate realized PnL
#=================================================

def update_positions(stock_price, quantity, action):
    
    """
    Updates all the current open positions, settling long positions if shares are sold or short positions if shares are bought.
    The settlement starts with the oldest long or short positions using the FIFO method.
    The oldest batch of shares and its price is compared with the actual batch and price, settling one old batch after another until there are
    no more shares remaining in the current batch or all the old batches have been settled.

    Once the settling has been performed, all the realized PnL for all the batch settlements are aggregated to update the total cumulative PnL.

    Also, the list of positions is updated for future settlings, adding the latest transaction if there are remaining shares after the settlement.

    Parameters:
    - stock_price (float): current market price of the stock.
    - quantity (int): number of shares bought/sold.
    - action (str): type of transaction performed, either "buy" or "sell".

    """

    global positions, total_cum_pnl

    remaining_shares = quantity
    realized_pnl = 0 
    
    if action == 'buy':
        for index, row in positions.iterrows():
            if row["Short"]>0 and remaining_shares >0:
                shares_sold = min(row["Short"], remaining_shares)
                
                positions.loc[index, "Short"] -= shares_sold
                remaining_shares -= shares_sold

                realized_pnl += (row["Stock price"]-stock_price)*shares_sold
            
        if remaining_shares >0:
            new_entry = pd.DataFrame([{"Stock price": stock_price, "Long": remaining_shares, "Short": 0}])
            positions = pd.concat([positions, new_entry], ignore_index=True) if not positions.empty else new_entry 

    elif action == 'sell':
        for index, row in positions.iterrows():
            if row["Long"]>0 and remaining_shares >0:
                shares_sold = min(row["Long"], remaining_shares)
                
                positions.loc[index, "Long"] -= shares_sold
                remaining_shares -= shares_sold

                realized_pnl += (stock_price-row["Stock price"])*shares_sold
       
        if remaining_shares >0:
            new_entry = pd.DataFrame([{"Stock price": stock_price, "Long": 0, "Short": remaining_shares}])
            positions = pd.concat([positions, new_entry], ignore_index=True) if not positions.empty else new_entry
    
    return realized_pnl

# ================================================
# 4. Function to calculate  unrealized PnL
#=================================================

def calculate_upnl(stock_price, quantity, action):

    """
    Calculates the unrealized PnL.

    It compares all open positions (number of shares and price) with the current stock price, calculating the PnL for every position
    or batch of shares.

    Parameters:
    - stock_price (float): Current market price of the stock.
    - quantity (int): Number of shares bought/sold.
    - action (str): tyoe of transaction performed, either "buy" or "sell".
    
    """
    global positions
    
    unrealized_pnl_long = sum((stock_price-row["Stock price"])*row["Long"] 
                              for _, row in positions.iterrows())

    unrealized_pnl_short = sum((row["Stock price"]-stock_price)*row["Short"]
                               for _, row in positions.iterrows())

    unrealized_pnl = unrealized_pnl_long + unrealized_pnl_short

    return unrealized_pnl
    
# ================================================
# 5. Function to show a table
#=================================================

def show_results():

    """
    Shows the output in an HTML scrollable table compatible with Jupyter Notebooks
    
    """

    trades_table = pd.DataFrame(trades)

    html_table = trades_table.to_html(index=False, classes="table table-bordered")
    scrollable_output = f'<div style="overflow-x: auto; white-space: nowrap;">{html_table}</div>'
    
    display(HTML(scrollable_output))

# ================================================
# 6. Running the script requesting inputs
#=================================================

    """
    Runs the script asking the user for inputs and showing the calculations in a table output
    
    """

while True:
    try:
        action = input("Insert the operation ('buy' or 'sell')").strip().lower()
        if action not in ["buy", "sell"]:
            print("Invalid input for action: please insert 'buy' or 'sell'")
            continue
        stock_price = float(input('Insert stock price'))
        quantity = float(input('Insert quantity of shares'))
        
        process_trade(stock_price, quantity, action)
    
        show_results()
    
        add_trade = input("Do you want to add another trade? ('yes' or 'no')").strip().lower()
        
        if add_trade == "no":
            print('Ending trades')
            break
    
    except ValueError:
            print("Invalid input: Please enter a valid number.")

while True:
    try:
        action = input("Do you want to calculate interest earnt or lost? ('yes' or 'no'):").strip().lower()
        if action not in ["yes", "no"]:
            print("Invalid input: please insert 'yes' or 'no':")
            continue

        if action == "no":
            break
        else:
            i = float(input("Please insert annual interest rate:").strip())
            c = float(Fraction(input("Please insert compounding period in fraction form (1/12, 18/12 etc.):".strip())))
            total_interest = sum(((trade["Cumulative cash flow"]) * i * c)
                              for trade in trades)
            real_total_cum_pnl = total_cum_pnl + total_interest

            print(f"The total interest generated is {total_interest:.2f}. "
                  f"The real total cumulative PnL is {real_total_cum_pnl:.2f}.")
            break
    
    except ValueError:
            print("Invalid input: Please enter a valid number.") 


Insert the operation ('buy' or 'sell') buy
Insert stock price 46.78
Insert quantity of shares 3000


Trade sequence,Stock price,Trade,Open share position,Cash flow credit (+) or debit (-),Cumulative cash flow,Realized PnL,Unrealized PnL,Total cumulative profit (+) or loss (-)
1,46.78,Buy 3000.0,Long 3000.0,-140340.0,-140340.0,0,0.0,0.0


Do you want to add another trade? ('yes' or 'no') no


Ending trades


Do you want to calculate interest earnt or lost? ('yes' or 'no'): yes
Please insert annual interest rate: 0.06
Please insert compounding period in fraction form (1/12, 18/12 etc.): 1/12


The total interest generated is -701.70. The real total cumulative PnL is -701.70.


Do you want to calculate interest earnt or lost? ('yes' or 'no'): no
